In [1]:
import torch
import torch.nn as nn
from torchsummary import summary


class IdentityBlock(nn.Module):
    def __init__(self, in_channels, filters):
        super(IdentityBlock, self).__init__()

        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Conv2d(in_channels, filters[0], 1)
        self.bn1 = nn.BatchNorm2d(filters[0])

        self.conv2 = nn.Conv2d(filters[0], filters[1], 3, padding=1)
        self.bn2 = nn.BatchNorm2d(filters[1])

        self.conv3 = nn.Conv2d(filters[1], in_channels, 1)
        self.bn3 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        x_shortcut = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn2(x)

        x = self.conv3(x)
        x = self.bn3(x)

        x += x_shortcut
        x = self.relu(x)

        return x


class ConvolutionalBlock(nn.Module):
    def __init__(self, in_channels, filters, stride):
        super(ConvolutionalBlock, self).__init__()
        
        self.relu = nn.ReLU()

        self.conv1 = nn.Conv2d(in_channels, filters[0], kernel_size=1, stride=stride)
        self.bn1 = nn.BatchNorm2d(filters[0])
        
        self.conv2 = nn.Conv2d(filters[0], filters[1], kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(filters[1])
        
        self.conv3 = nn.Conv2d(filters[1], filters[2], kernel_size=1)
        self.conv_shortcut = nn.Conv2d(in_channels, filters[2], kernel_size=1, stride=stride)
        self.bn3 = nn.BatchNorm2d(filters[2])

    def forward(self, x):
        x_shortcut = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        
        x_shortcut = self.conv_shortcut(x_shortcut)
        x_shortcut = self.bn3(x_shortcut)
        
        x += x_shortcut
        x = self.relu(x)
        
        return x

class ResNet(nn.Module):
    def __init__(self, layers, block, num_classes):
        super(ResNet, self).__init__()

        self.num_classes = num_classes

        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.max_pool = nn.MaxPool2d(3, 2)

        self.layer1 = self._get_layer(layers[0], block, 64, [64, 64, 256], 1)
        self.layer2 = self._get_layer(layers[1], block, 256, [128, 128, 512], 2)
        self.layer3 = self._get_layer(layers[2], block, 512, [256, 256, 1024], 2)
        self.layer4 = self._get_layer(layers[3], block, 1024, [512, 512, 2048], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    @staticmethod
    def _get_layer(n, block, in_channels, filters, stride):
        layers = [ConvolutionalBlock(in_channels, filters, stride)]
        for i in range(n):
            layers.append(block(filters[2], filters[:2]))

        return nn.Sequential(*layers)

In [2]:
torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# classes, train, val, test = get_datasets()

model = ResNet(num_classes=101, block=IdentityBlock, layers=[2, 3, 5, 2]).to(device)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 109, 109]           9,472
       BatchNorm2d-2         [-1, 64, 109, 109]             128
              ReLU-3         [-1, 64, 109, 109]               0
         MaxPool2d-4           [-1, 64, 54, 54]               0
            Conv2d-5           [-1, 64, 54, 54]           4,160
       BatchNorm2d-6           [-1, 64, 54, 54]             128
              ReLU-7           [-1, 64, 54, 54]               0
            Conv2d-8           [-1, 64, 54, 54]          36,928
       BatchNorm2d-9           [-1, 64, 54, 54]             128
             ReLU-10           [-1, 64, 54, 54]               0
           Conv2d-11          [-1, 256, 54, 54]          16,640
      BatchNorm2d-12          [-1, 256, 54, 54]             512
           Conv2d-13          [-1, 256, 54, 54]          16,640
      BatchNorm2d-14          [-1, 256,